In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import tkinter as tk
from tkinter import filedialog
from tkinter import Label
from PIL import ImageTk, Image

# 設置數據集路徑
base_dir = r'C:\dataFINAL\archive\Garbage classification\Garbage classification'
train_dir = base_dir
validation_dir = base_dir  # 假設數據集沒有分為訓練和驗證集，使用同一個目錄

# 圖像數據生成器和數據增強
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 使用 20% 數據作為驗證集
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# 使用預訓練模型 VGG16
base_model = tf.keras.applications.VGG16(include_top=False, input_shape=(150, 150, 3))

# 鎖定預訓練模型的權重
for layer in base_model.layers:
    layer.trainable = False

# 模型構建
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # 假設有6個分類
])
    
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 模型訓練
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50  # 增加訓練次數
)

# 模型評估
validation_generator.reset()
Y_pred = model.predict(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = list(validation_generator.class_indices.keys())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

# 保存模型
model.save('garbage_classification_model_vgg16.h5')



Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.
Epoch 1/50


C:\Users\Kevin\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.2285 - loss: 1.9568 - val_accuracy: 0.4333 - val_loss: 1.4739
Epoch 2/50
 1/63 ━━━━━━━━━━━━━━━━━━━━ 51s 823ms/step - accuracy: 0.4062 - loss: 1.5515

C:\Users\Kevin\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4062 - loss: 1.5515 - val_accuracy: 0.6087 - val_loss: 1.3323
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.4001 - loss: 1.4937 - val_accuracy: 0.4875 - val_loss: 1.3216
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.3750 - loss: 1.3337 - val_accuracy: 0.6087 - val_loss: 1.0401
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.4543 - loss: 1.3549 - val_accuracy: 0.4979 - val_loss: 1.2396
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5312 - loss: 1.1558 - val_accuracy: 0.5652 - val_loss: 1.2712
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.4903 - loss: 1.3032 - val_accuracy: 0.5625 - val_loss: 1.1858
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6250 - loss: 1.1834 - val_accuracy: 0.5652 - val_loss: 1.2539
Epoch 9/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.5116 - loss: 1.2224 - val_accuracy: 0.5604 - val_loss: 1.1033

Confusion Matrix
[[11 27 10 20 12  0]
 [18 21 14 31 16  0]
 [11 24 15 20 10  2]
 [23 24 20 32 18  1]
 [20 33 18 14  7  4]
 [ 8  4  5  7  3  0]]
Classification Report
              precision    recall  f1-score   support

   cardboard       0.12      0.14      0.13        80
       glass       0.16      0.21      0.18       100
       metal       0.18      0.18      0.18        82
       paper       0.26      0.27      0.26       118
     plastic       0.11      0.07      0.09        96
       trash       0.00      0.00      0.00        27

    accuracy                           0.17       503
   macro avg       0.14      0.15      0.14       503
weighted avg       0.16      0.17      0.16       503



In [ ]:
import tkinter as tk
from tkinter import filedialog, Label, Frame, Button
from PIL import ImageTk, Image
import numpy as np
import tensorflow as tf

# 讀取模型
model = tf.keras.models.load_model('garbage_classification_model_vgg16.h5')
target_names = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

# 創建主窗口
root = tk.Tk()
root.title("垃圾分類系統")
root.geometry("600x400")
root.configure(bg="#f0f0f0")

# 標題
title_label = tk.Label(root, text="垃圾分類系統", font=("Helvetica", 24, "bold"), bg="#f0f0f0")
title_label.pack(pady=20)

# 上傳圖片框架
frame = Frame(root, bg="#f0f0f0")
frame.pack(pady=10)

# 圖片顯示區域
img_label = Label(frame, bg="#f0f0f0")
img_label.pack(pady=10)

# 上傳按鈕
upload_button = Button(frame, text="上傳圖片", command=lambda: upload_image(frame), font=("Helvetica", 14), bg="#4CAF50", fg="white")
upload_button.pack(pady=10)

# 結果顯示標籤
result_label = Label(root, text="", font=("Helvetica", 16), bg="#f0f0f0")
result_label.pack(pady=20)

def upload_image(frame):
    file_path = filedialog.askopenfilename()
    if not file_path:
        return
    
    uploaded = Image.open(file_path)
    uploaded.thumbnail((150, 150))
    img = ImageTk.PhotoImage(uploaded)
    img_label.config(image=img)
    img_label.image = img

    img_array = np.array(uploaded.resize((150, 150))) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    predicted_class = target_names[np.argmax(prediction)]
    result_label.config(text=f"分類結果: {predicted_class}", fg="#333")

# 版權信息
footer_label = Label(root, text="© 2024 垃圾分類系統", font=("Helvetica", 10), bg="#f0f0f0", fg="#888")
footer_label.pack(side=tk.BOTTOM, pady=10)

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
